In [ ]:
import pandas as pd
import numpy as np

## Data Collection

### Files sources

In [ ]:
import requests 

In [ ]:
# URLs of the files
train_datas_url = 'https://www.raphaelcousin.com/modules/data-science-practice/module4/exercise/module4_exercise_train.zip'
test_data_url = 'https://www.raphaelcousin.com/modules/data-science-practice/module4/exercise/Neighborhood_Market_data.csv'

# Function to download a file
def download_file(url, file_name):
    response = requests.get(url)
    response.raise_for_status()  # Ensure we notice bad responses
    with open(file_name, 'wb') as file:
        file.write(response.content)
    print(f'Downloaded {file_name} from {url}')

# Downloading the files
download_file(train_datas_url, 'module4_exercise_train.zip')
download_file(test_data_url, 'Neighborhood_Market_data.csv')

#### CityMart

In [ ]:
!unzip "module4_exercise_train.zip"

In [ ]:
# read "CityMart_data.csv
df_cm = pd.read_csv('CityMart_data.csv')



#### Greenfield_Grocers

In [ ]:
# read "Greenfield_Grocers_data.csv"
df_gg = pd.read_csv('Greenfield_Grocers_data.csv',sep='|')

In [ ]:
df_gg

In [ ]:
df_gg.drop(df_gg.index[0:2], inplace=True)


In [ ]:
df_gg.columns = df_gg.iloc[0].str.lower()
df_gg = df_gg[1:].reset_index(drop=True)  # supprimer la première ligne et réindexer

In [ ]:
df_gg = df_gg.iloc[:,:-2]

In [ ]:
df_gg

#### Outlet_data

In [ ]:
!pip install openpyxl

In [ ]:
# read "SuperSaver_Outlet_data.xlsx"
df_ss = pd.read_excel("SuperSaver_Outlet_data.xlsx")

#### HighStreet_Bazaar

In [ ]:
# read 'HighStreet_Bazaar_data.json'
df_hb = pd.read_json('HighStreet_Bazaar_data.json')

#### Aggregate

#### Simple baseline

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error, accuracy_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import numpy as np

def get_simple_baseline(data, fillna_value=-1, drop_cols=None, k_fold=5, scaler='standard', model='linear', metric='mae', target_col=None, X_data_test=None):
    
    data = data.copy()
    # Handle missing values
    data.fillna(fillna_value, inplace=True)
    if X_data_test is not None:
        X_data_test = X_data_test.copy()
        X_data_test.fillna(fillna_value, inplace=True)
    
    # Drop unwanted columns
    if drop_cols:
        data.drop(drop_cols, axis=1, inplace=True)
        if X_data_test is not None:
            X_data_test.drop(drop_cols, axis=1, inplace=True)

    # Split data into features (X) and target (y)
    y = data[target_col]
    X = data.drop(target_col, axis=1)

    # Feature scaling
    if scaler == 'standard':
        scaler = StandardScaler()
    elif scaler == 'minmax':
        scaler = MinMaxScaler()
    else:
        scaler = None
    
    if scaler:
        X = scaler.fit_transform(X)
        if X_data_test is not None:
            X_data_test = scaler.transform(X_data_test)

    # Initialize the model
    if model == 'linear':
        model = LinearRegression()
    elif model == 'logistic':
        model = LogisticRegression()
    elif model == 'random_forest':
        model = RandomForestClassifier()
    else:
        raise ValueError("Unsupported model type")

    # Initialize cross-validation
    kf = KFold(n_splits=k_fold, shuffle=True, random_state=42)
    scores = []

    # Train and evaluate using k-fold cross-validation
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        # Train the model
        model.fit(X_train, y_train)
        
        # Make predictions
        y_pred = model.predict(X_test)
        
        # Evaluate using the specified metric
        if metric == 'mae':
            score = mean_absolute_error(y_test, y_pred)
        elif metric == 'accuracy':
            score = accuracy_score(y_test, np.round(y_pred))
        else:
            raise ValueError("Unsupported metric")

        scores.append(score)

    if X_data_test is not None:
        model.fit(X, y)
        return np.mean(scores), model.predict(X_data_test)
    
    # Return the average score
    return np.mean(scores),y_pred

In [ ]:
data = pd.concat(
    [df_gg, df_cm,df_hb,df_ss],
    axis=0,                      # 0 for vertical, 1 for horizontal
    ignore_index=True) 

In [ ]:
get_simple_baseline(data, fillna_value=-1, k_fold=5,drop_cols=['item_code','store_name','last_modified'] ,scaler='standard', model='linear', metric='mae', target_col='quantity_sold')

### API sources

In [ ]:
def get_api(endpoint_url):
    try:
        # Make the GET request to the mock API
        response = requests.get(endpoint_url)

        if response.status_code == 200:
            data = response.json()
            print(data["message"])
            return data['data']
        else:
            print(f"Failed to retrieve volume data. Status code: {response.status_code}")
    
    except Exception as e:
        print(f"An error occurred: {e}")
password =get_api('https://www.raphaelcousin.com/api/exercise/auth') 
password = password['password']
print(password)

prices = get_api(f'https://www.raphaelcousin.com/api/exercise/{password}/prices')

In [ ]:

df_prices = pd.DataFrame(list(prices.items()), columns=["item_code", "prices"])

In [ ]:
df_prices

#### Aggregate

In [ ]:
data1 = pd.merge(data, df_prices, on = 'item_code',how="inner")
#data1 = pd.merge(data, df_prices, left_index=True, right_index=True, how='left')
#data1 = data1.dropna()

In [ ]:
data1.head()

In [ ]:
get_simple_baseline(data1, fillna_value=-1, drop_cols=['item_code','store_name','last_modified'], k_fold=5, scaler='standard', model='linear', metric='mae', target_col='quantity_sold')

### Scrapping sources

In [ ]:
!pip install selenium

In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import time

# Set up the Selenium WebDriver (e.g., Chrome)
#driver = webdriver.Chrome()  # Make sure ChromeDriver is installed
driver = webdriver.Firefox()
# driver = webdriver.Edge()
# driver = webdriver.Safari()

# Open the URL
url = 'https://www.raphaelcousin.com/module4/scrapable-data'
driver.get(url)

# Wait for the page to fully load (increase time if needed)
time.sleep(5)

# Get the fully rendered page source
html = driver.page_source

# Parse the HTML with BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

# Initialize lists to store scraped data
exercise_data = []

# Find both tables
tables = soup.find_all('table')

# Close the Selenium WebDriver
driver.quit()

# Scrape the second table (Exercise Data)
course_table = tables[1]

course_table = tables[1]
for row in course_table.find('tbody').find_all('tr'):
    cols = row.find_all('td')
    x = [col.get_text(strip=True) for col in cols]    
    exercise_data.append(x)

print(exercise_data)
# Construire le DataFrame avec toutes les colonnes



In [ ]:
# Convert the lists to pandas DataFrames
df_exercise = pd.DataFrame(exercise_data)
df_exercise
df_exercise.rename(columns={0: "item_code",1:"1",2:"2",3:"3"}, inplace=True)
df_exercise = df_exercise.astype({"1":float,"2":float,"3":float})   



In [ ]:
df_exercise

#### Aggregate

In [ ]:
data2 = pd.merge(data1, df_exercise, on='item_code',  how='inner')

In [ ]:
data2

In [ ]:
get_simple_baseline(data2, fillna_value=-1, k_fold=5, drop_cols= ['item_code','store_name','last_modified'], scaler='standard', model='linear', metric='mae', target_col='quantity_sold')

### Generating Submission File

In [ ]:


df_StoreN =  pd.read_csv("Neighborhood_Market_data.csv", sep=",", index_col='item_code')
df_StoreN

In [ ]:
#df_StoreN = pd.merge(df_StoreN, df_prices, left_index=True, right_index=True, how='left')
#df_StoreN = pd.merge(df_StoreN, df_exercise, left_index=True, right_index=True, how='left')
df_StoreN = pd.merge(df_StoreN, df_prices, on = 'item_code', how='inner')
df_StoreN = pd.merge(df_StoreN, df_exercise, on ='item_code', how='inner')

In [ ]:
df_StoreN.columns

In [ ]:
df_StoreN

In [ ]:
_, x_pred = get_simple_baseline(data2, fillna_value=-1, drop_cols=['item_code','store_name','last_modified'], k_fold=5, scaler='standard',target_col=['quantity_sold'], model='linear', metric='mae', X_data_test = df_StoreN)

In [ ]:
print(x_pred.flatten())

In [ ]:
submission = pd.DataFrame({
    'item_code': df_StoreN.index,
    'quantity_sold': x_pred.flatten() # your_prediction
})

submission.to_csv('submission.csv', index=False, sep=',')
submission.head()